In [1]:
import math
from interval import interval, imath
from Polynomials import *

In [57]:
class Taylor:
    def __init__(self, poly:Poly, order:int, remainder:interval, domain:interval):
        self.P=poly
        self.order=order
        self.I=interval(remainder)
        self.D=interval(domain)

    def __repr__(self):
        return 'TaylorModel({0}, {1}, {2}, {3} )'.format(self.P, self.order, self.I, self.D)
    
    def __add__(self, other):
        if isinstance(other,(int,float)):
            other=Taylor(Poly([other],0),0,0,self.D)
        return Taylor(self.P+other.P,(self.P+other.P).order,self.I+other.I,self.D)
    
    def __radd__(self,other):
        return self+other
    
    def __sub__(self, other):
        if isinstance(other,(int,float)):
            other=Taylor(Poly([other],0),0,0,self.D)
        return Taylor(self.P-other.P,(self.P-other.P).order,self.I-other.I,self.D)
    
    def __rsub__(self,other):
        if isinstance(other,(int,float)):
            other=Taylor(Poly([other],0),0,0,self.D)
        return other-self
    
    def __mul__(self, other):
        if isinstance(other,(int,float)):
            if other==0:
                return 0
            else:
                return Taylor(other*self.P,self.order,other*self.I,self.D)
        order=max(self.order,other.order)
        P=Taylor(self.P*other.P,order,self.I,self.D).truncation(order)
        I1=(Taylor(self.P*other.P,order,self.I,self.D)-P).bound_poly()
        I2=self.bound_poly()*other.I
        I3=self.I*other.bound_poly()
        I4=self.I*other.I
        return Taylor(P.P,order,I1+I2+I3+I4,self.D)
    
    def __rmul__(self,other):
        return self*other
    
    #bound polynomials
    def bound_poly(self):
        B=self.P.coef[0]
        n=self.P.order
        for i in range(1,n+1):
            B+=interval(self.P.coef[i])*(self.D**i)
        return B
    
    def truncation(self,n):
        return Taylor(Poly(self.P.coef[:n+1],n),n,self.I,self.D)
    
    def __truediv__(self, other):
        if isinstance(other,(int,float)):
            if other==0:
                raise Exception("Cannot dividied by zero!")
            else:
                return Taylor(self.P/other,self.order,self.I/other,self.D)
        else:
            raise Exception("Can only do scalar division!")
    
    def one(self):
        return Taylor(Poly.one(self.order),self.order,self.I,self.D)
    
    def zero(self):
        return Taylor(Poly.zero(self.order),self.order,self.I,self.D)
    
    def square(self):
        return self*self
    
    def __pow__(self,n):
        if n==0:
            return self.one()
        elif n%2==0:
            if n==2:
                return self.square()
            else:
                p=int(n/2)
                return self.__pow__(p).square()
        elif n%2==1:
            if n==1:
                return self
            else:
                p=int((n-1)/2)
                return self*self.__pow__(p).square()
        else:
            raise Exception("n must be a positive integer")
    
    #composition of a function 'f' and a Taylor model 'self'
    def compose(f,self):
        pass
            

In [25]:
a=Taylor(Poly([0,1,2,3,4],4),4,interval([0,1]),interval([-1,1]))
b=Taylor(Poly([1,2,3,4,5],4),4,interval([0,2]),interval([-1,1]))
a

TaylorModel(Poly([0, 1, 2, 3, 4], 4), 4, interval([0.0, 1.0]), interval([-1.0, 1.0]) )

In [26]:
b

TaylorModel(Poly([1, 2, 3, 4, 5], 4), 4, interval([0.0, 2.0]), interval([-1.0, 1.0]) )

In [27]:
a+b

TaylorModel(Poly([1, 3, 5, 7, 9], 4), 4, interval([0.0, 3.0]), interval([-1.0, 1.0]) )

In [28]:
1+a

TaylorModel(Poly([1, 1, 2, 3, 4], 4), 4, interval([0.0, 1.0]), interval([-1.0, 1.0]) )

In [29]:
a-a

TaylorModel(Poly([0, 0, 0, 0, 0], 4), 4, interval([-1.0, 1.0]), interval([-1.0, 1.0]) )

In [30]:
1-a

TaylorModel(Poly([1, -1, -2, -3, -4], 4), 4, interval([-1.0, 0.0]), interval([-1.0, 1.0]) )

In [31]:
interval([-1,1])+2*(interval([-1,1]))**2+3*(interval([-1,1]))**3+4*(interval([-1,1]))**4

interval([-4.0, 10.0])

In [32]:
a.P(interval([-1,1]))

interval([-10.0, 10.0])

In [33]:
a.bound_poly()

interval([-4.0, 10.0])

In [34]:
a

TaylorModel(Poly([0, 1, 2, 3, 4], 4), 4, interval([0.0, 1.0]), interval([-1.0, 1.0]) )

In [35]:
a-a.truncation(3)

TaylorModel(Poly([0, 0, 0, 0, 4], 4), 4, interval([-1.0, 1.0]), interval([-1.0, 1.0]) )

In [36]:
P1=Taylor(a.P*b.P,max(a.order,b.order),a.I,a.D)
P1

TaylorModel(Poly([0, 1, 4, 10, 20, 30, 34, 31, 20], 8), 4, interval([0.0, 1.0]), interval([-1.0, 1.0]) )

In [37]:
P=Taylor(a.P*b.P,max(a.order,b.order),a.I,a.D).truncation(max(a.order,b.order))
P

TaylorModel(Poly([0, 1, 4, 10, 20], 4), 4, interval([0.0, 1.0]), interval([-1.0, 1.0]) )

In [38]:
P1.bound_poly()

interval([-72.0, 150.0])

In [39]:
P.bound_poly()

interval([-11.0, 35.0])

In [40]:
P1.bound_poly()-P.bound_poly()

interval([-107.0, 161.0])

In [41]:
P1-P

TaylorModel(Poly([0, 0, 0, 0, 0, 30, 34, 31, 20], 8), 8, interval([-1.0, 1.0]), interval([-1.0, 1.0]) )

In [42]:
I1=(P1-P).bound_poly()
I1

interval([-61.0, 115.0])

In [43]:
(Taylor(a.P*b.P,max(a.order,b.order),a.I,a.D)-P).P(interval([-1.0, 1.0]))

interval([-115.0, 115.0])

In [44]:
I2=a.bound_poly()*b.I
I2

interval([-8.0, 20.0])

In [45]:
I3=b.bound_poly()*a.I
I3

interval([-5.0, 15.0])

In [46]:
I4=a.I*b.I
I4

interval([0.0, 2.0])

In [47]:
I1+I2+I3+I4

interval([-74.0, 152.0])

In [48]:
a*b

TaylorModel(Poly([0, 1, 4, 10, 20], 4), 4, interval([-74.0, 152.0]), interval([-1.0, 1.0]) )

In [49]:
a*2

TaylorModel(Poly([0, 2, 4, 6, 8], 4), 4, interval([0.0, 2.0]), interval([-1.0, 1.0]) )

In [50]:
a/2

TaylorModel(Poly([0.0, 0.5, 1.0, 1.5, 2.0], 4), 4, interval([0.0, 0.5]), interval([-1.0, 1.0]) )

In [51]:
Poly.one(5)

Poly([1, 0, 0, 0, 0, 0], 5)

In [52]:
a*a*a*a*a

TaylorModel(Poly([0, 0, 0, 0, 0], 4), 4, interval([-78148.0, 161051.0]), interval([-1.0, 1.0]) )

In [53]:
a**5

TaylorModel(Poly([0, 0, 0, 0, 0], 4), 4, interval([-78764.0, 161051.0]), interval([-1.0, 1.0]) )

In [54]:
interval([-1,1])**2+interval([-1,1])**3+interval([-1,1])**4+interval([0,2])+interval([0,1])*interval([-1,1])+interval([-1,3])

interval([-3.0, 9.0])

In [55]:
Taylor(Poly([1,1,0,1],3),3,interval([0,1]),interval([-1,1])).bound_poly()

interval([-1.0, 3.0])

In [56]:
Poly([1,1,0,1],3)(interval([-1,1]))

interval([-1.0, 3.0])